# GPT를 활용해 리뷰요약 best prompt 작성
- 우리 서비스에서 사용할 GPT모델 : gpt3.5-turbo

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 절대경로나 정확한 상대경로 지정
load_dotenv('C:/Users/Admin/kpmg_future_lab/LLM_ex/yanolja_summary/.env', override=True)

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
print(f"API Key loaded: {OPENAI_API_KEY[:10]}...")  # 앞 10글자만 출력 (확인용)

# OpenAI 클라이언트 생성
client = OpenAI(api_key=OPENAI_API_KEY)

# Chat API 호출
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

# 응답 출력
print(response.choices[0].message.content)


API Key loaded: sk-proj-vm...
Hello! How can I assist you today?


# 데이터 전처리 
- 리뷰 데이터 파일 로딩 
- 리뷰 요약 기준 (평점 1-5)
    - good : 평점 5
    - bad : 평점 4이하 
- 각 리뷰마다 스페셜 토큰 
    - '[REVIEW_START]' + r['review'] + '[REVIEW_END]'


## 데이터 전처리 함수 작성 

In [14]:
import json

with open('./res/3.kyongpo_the_hongc.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다',
  'stars': 5,
  'date': '2025.02.03'},
 {'review': '로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다',
  'stars': 5,
  'date': '2024.12.23'},
 {'review': '친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요',
  'stars': 5,
  'date': '2024.12.09'}]

### 좋은 평점 및 나쁜 평점 정의 코드

In [15]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(329, 91)

### 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장 

In [16]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        # specital token 추가
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        # specital token 추가
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_good[:3]

['[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]',
 '[REVIEW_START]로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다[REVIEW_END]',
 '[REVIEW_START]친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요[REVIEW_END]']

### 하나의 문자열로 합치기 

In [17]:
# good 문자, 하나의 문자열로 합치기
reviews_good_text = '\n'.join(reviews_good)
reviews_good_text[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

In [18]:
# bad 문자, 하나의 문자열로 합치기
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text[:100]

'[REVIEW_START]방음이 좀 안되긴 한데 욕실 깨끗해서 좋아요[REVIEW_END]\n[REVIEW_START]공사한다고 예약힐때부터 알려주면좋겠고 체크인할때 공사한다고 불편'

- 하나의 숙박시설에 대한 리뷰 데이터 전처리 함수 정의 

In [ ]:
# 정의부 
import datetime
from dateutil import parser

def preprocess_reviews(data_path):

    with open(data_path, 'r', encoding='utf-8') as f:
        # list 형태로 json 파일 열기
        review_list = json.load(f)
    
    # good, bad 데이터 저장 변수 초기화
    reviews_good, reviews_bad = [], []
    # 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)

    # 리뷰데이터 
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date
        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue
        # 평점 5점 기준
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 하나의 문자열 리스트로 합치기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)# good 리뷰와 bad 리뷰 반환
    return reviews_good_text, reviews_bad_text


In [39]:
# 호출부 
data_path = "res/1.gangneung_mare.json"
preprocess_reviews(data_path)
# print(preprocess_reviews(data_path))
good, bad = preprocess_reviews(data_path)
good[:100]


'[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_E'

In [36]:
# 날짜 정리 
# import datetime
# current_date = datetime.datetime.now()
# current_date - datetime.timedelta(days=6*30)

datetime.datetime(2024, 12, 6, 12, 34, 6, 11136)

In [40]:
bad[:100]

'[REVIEW_START]우선 바람이 불어서 예정보다 30분 숙소 입실을 허락해줌\n그런데 오션뷰 2층 205호는 앞에 가로수들때문에 오션전혀 기대하시면 안됩니다. 그리고 침구는 발'

### 평가용 함수 작성 
#### 평가 기준 설정 
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 vs. 점수 매기는 방식
    - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

#### 평가 스크립트 작성 
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)
    - [[A]] – 어시스턴트 A가 더 나을 경우
    - [[B]] – 어시스턴트 B가 더 나을 경우
    - [[C]] – 두 응답이 동등한 품질일 경우

In [41]:
# def pairwise_eval(리뷰원본, AI_a, AI_b):
def pairwise_eval(reviews, answer_a, answer_b):
    # 평가 프롬프트 
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
    
    # OpenAI API를 사용하여 평가 요청
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion.choices[0].message.content

### Baseline 응답 개발 

In [42]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [47]:
# 높은 평점만 리뷰 데이터만 받음음
reviews, _ = preprocess_reviews(data_path='./res/3.kyongpo_the_hongc.json')

### 리뷰 데이터 요약 함수 정의 및 실행

In [48]:
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion.choices[0].message.content

print(summarize(reviews, PROMPT_BASELINE))

강릉 여행 중 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔해서 만족스러웠고, 침구가 편안해서 잘 수 있었다. 친절한 안내와 청결한 환경으로 편안하게 머물렀고, 위치도 좋아서 편안한 휴식을 즐겼다. 부모님도 만족하셨고, 직원들의 친절함과 시설의 깨끗함이 좋았다.


### 응답결과 평가하기 
대조군 문장 정의하기

In [49]:
# 대조군 문장 가져오기(야놀자 AI good 요약 문장)
# summary_real_20250530 = '숙소는 강릉 경포해변 바로 앞에 위치하며, 오션뷰와 경포호수 전망을 제공합니다. 객실과 화장실이 청결하게 관리되고 있으며, 편안한 침대와 고급 침구로 쾌적한 숙박 환경을 제공합니다. 주변에 식당과 편의시설이 가까워 접근성이 좋고, 주차 공간도 넓습니다. 직원들의 친절한 응대와 서비스도 긍정적인 평가를 받고 있으며, 대체로 소음 차단이 잘 이루어져 조용한 환경에서 휴식을 취할 수 있습니다. 전반적으로 방문자들은 숙소의 청결도와 편안함, 아늑한 분위기에 매우 만족하는 후기를 남기고 있습니다.'

summary_4o_20250530 = summarize(reviews, PROMPT_BASELINE, temperature=0.0, model='gpt-4o-2024-05-13')
summary_4o_20250530

'이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게 이용할 수 있습니다. 다만, 배달 음식 제한과 방음 문제는 일부 아쉬운 점으로 지적되었습니다. 전반적으로 재방문 의사가 많고 추천할 만한 숙소로 평가받고 있습니다.'

In [50]:
# 비교하기
# pairwise_eval(리뷰 데이터, AI 요약한 데이터_A, 대조군 AI 문장_B))
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE), summary_4o_20250530))

Both Assistant A and Assistant B provided summaries of the user reviews, but there are notable differences in their approaches.

**Assistant A's Summary:**
- Focuses on the positive aspects of the hotel, such as the owner's quality, quietness, luxurious lobby and rooms, comfortable bedding, friendly staff, and cleanliness.
- Mentions that the location is good and that parents were satisfied.
- Overall, the summary is positive but lacks mention of any negative points or areas for improvement.

**Assistant B's Summary:**
- Highlights the cleanliness, comfortable bedding, and friendly staff as positive aspects.
- Also mentions the luxurious lobby and good location with well-developed infrastructure.
- Points out some negative aspects, such as the restriction on delivery food and soundproofing issues.
- Concludes with a general positive note about the high likelihood of revisits and recommendations.

**Evaluation:**
- **Helpfulness:** Both summaries are helpful, but Assistant B provides a 

두 어시스턴트(A와 B)는 사용자 리뷰를 요약했지만, 접근 방식에 있어 뚜렷한 차이가 있습니다.  

어시스턴트 A의 요약:  

호텔의 긍정적인 면에 초점을 맞추고 있음. 예를 들어, 좋은 주인, 조용한 분위기, 고급스러운 로비와 객실, 편안한 침구, 친절한 직원, 청결함 등을 언급함.  

위치가 좋고, 부모님도 만족하셨다는 언급이 있음.  

전반적으로 긍정적인 요약이지만, 단점이나 개선할 부분은 언급하지 않음.  

어시스턴트 B의 요약:  

청결함, 편안한 침구, 친절한 직원 등 긍정적인 요소를 강조함.  

고급스러운 로비와 좋은 위치(잘 갖춰진 인프라 포함)도 언급함.  

배달 음식 제한, 방음 문제 등의 부정적인 요소도 함께 언급함.  

재방문 의사와 추천 가능성 등 긍정적인 결론으로 마무리함.  

평가:  

도움성: 두 요약 모두 도움이 되지만, 어시스턴트 B는 장단점을 모두 언급해 더 균형 잡힌 정보를 제공함.  

관련성: 두 요약 모두 사용자 리뷰와 관련이 높음.  

정확성: 두 요약 모두 리뷰 내용을 정확하게 반영함. 단, 어시스턴트 B는 단점도 언급하여 더 포괄적임.  

깊이: 어시스턴트 B는 장단점을 모두 포함하여 더 깊이 있는 분석을 제공함.  

창의성: 두 요약 모두 직설적이며 창의적인 요소는 부족함.  

세부사항: 어시스턴트 B는 배달 음식 제한, 방음 문제 등 구체적인 사항을 언급해 더 상세함.  

최종 결론:  
어시스턴트 B의 요약이 사용자 리뷰를 보다 균형 잡고 포괄적으로 반영하여 더 우수한 것으로 평가됨.  



## 대규모 평가 스크립트 
- 원래는 많은 수의 다양한 데이터에 대해 평가하지만 동일한 프롬프트에 대해 temperature 높여서 평가 진행  

In [52]:
# 같은 리뷰에 대해서 요약을 10번 수행(일반적으로는 50~100번 수행하면 좋음)
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0) for _ in range(eval_count)]
summaries_baseline

['강릉 여행 중 예약한 숙소는 주인의 품격이 느껴지며 조용하고 좋았고, 객실은 고급스럽고 깔끔하며 침구가 훌륭했어요. 직원들이 친절하고 안락한 환경에서 편안한 휴식이 가능했습니다. 위치가 좋고 깨끗하며 침구 상태가 좋았고, 부모님도 만족하셨습니다. 전체적으로 깨끗하고 친절하며 편안한 숙소였습니다.',
 '강릉 여행 중 예약한 호텔은 주인의 품격을 느낄 수 있는 조용하고 깨끗한 숙소였다. 고급 로비와 깔끔한 객실에서 편안히 쉬었으며, 친절한 안내와 청결한 시설로 만족했다. 위치가 좋고 침구상태도 좋아서 부모님도 만족하셨다. 전체적으로 만족스러운 서비스와 편안한 휴식을 즐긴 후 나감.',
 '강릉 여행길에 예약한 호텔은 주인의 품격이 느껴지는 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔해서 만족스러운 1박을 했어요. 직원들도 친절하고 청결한 숙소에서 편안히 쉴 수 있었어요. 위치도 좋고 침구 상태도 좋아서 잘 묵고 갑니다. 부모님도 만족하셔서 좋았고, 다음에도 이용할 의사가 있어요.',
 '강릉 여행길에 예약한 호텔은 주인의 품격이 느껴지고 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔하며 침구가 편안해서 만족스러웠다. 친절한 안내와 정리정돈이 잘 되어 있어 편안했고 위치도 좋아서 잘 쉬다 갔다. 숙소는 깨끗하고 이쁘며 침구 상태가 좋았고 직원들도 친절했다.체크인 시간보다 일찍 도착했는데 빠른 일처리에 감사하고 다음에도 이용할 의사가 있다고 했다.',
 '강릉 여행 중 예약한 호텔은 조용하고 편안하며 주인의 품격이 느껴져 만족스러웠다. 고급 로비와 깔끔한 객실에서 편안히 쉬고, 침구도 훌륭했다. 숙소는 깨끗하고 위치도 좋아서 편안한 숙박을 경험했으며, 직원들의 친절한 안내도 기억에 남는다. 부모님도 만족하셨고, 편안한 휴식을 즐길 수 있는 좋은 숙소였다.',
 '강릉 여행길에 예약을 잘한 호텔로 조용하고 품격있는 분위기, 깨끗하고 편안한 숙소로 만족했고, 친절한 안내와 정리정돈으로 편안한 휴식을 취할 수 있었으며, 위치가 좋아서 잘 쉬다 간다는 

In [53]:
pairwise_words = [summary_4o_20250530 for _ in range(len(summaries_baseline))]
pairwise_words

['이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게 이용할 수 있습니다. 다만, 배달 음식 제한과 방음 문제는 일부 아쉬운 점으로 지적되었습니다. 전반적으로 재방문 의사가 많고 추천할 만한 숙소로 평가받고 있습니다.',
 '이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게 이용할 수 있습니다. 다만, 배달 음식 제한과 방음 문제는 일부 아쉬운 점으로 지적되었습니다. 전반적으로 재방문 의사가 많고 추천할 만한 숙소로 평가받고 있습니다.',
 '이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게 이용할 수 있습니다. 다만, 배달 음식 제한과 방음 문제는 일부 아쉬운 점으로 지적되었습니다. 전반적으로 재방문 의사가 많고 추천할 만한 숙소로 평가받고 있습니다.',
 '이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게 이용할 수 있습니다. 다만, 배달 음식 제한과 방음 문제는 일부 아쉬운 점으로 지적되었습니다. 전반적으로 재방문 의사가 많고 추천할 만한 숙소로 평가받고 있습니다.',
 '이 숙소는 청결하고 깔끔한 객실과 편안한 침구로 많은 이용객들에게 좋은 평가를 받았습니다. 직원들의 친절한 서비스와 고급스러운 로비도 만족스러운 요소로 언급되었습니다. 위치가 좋고 주변 인프라가 잘 갖춰져 있어 편리하게

In [54]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        # 최종 결과
        verdict_text = pairwise_eval(reviews, answers_a[i], answers_b[i])

        # A인경우 카운트
        if '[[A]]' in verdict_text:
            a_cnt += 1
        # B인경우 카운트
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        # C인경우 카운트
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        # 나머지
        else:
            print('Evaluation Error')
    # a good, b bad, c 무승부 비슷함.
    return a_cnt, b_cnt, draw_cnt

# 요약한 것과 대조군 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_4o_20250530 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:43<00:00,  4.36s/it]

Wins: 0, Losses: 10, Ties: 0


### 응답결과 고도화 1 - 요약 조건들 명시 후 평가
- role과 수행 목표 정의
- 요약 결과에 대한 충족 조건
    - 톤엔 매너와 예시
    - 요약 결과에 갯수

In [55]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:36<00:00,  3.63s/it]

Wins: 6, Losses: 4, Ties: 0


### 응답결과 고도화 2 - 입력 데이터의 품질 증가
- 리뷰 길이가 짧은 것 필터링함, 고품질 리뷰만 남김

In [ ]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/3.kyongpo_the_hongc.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        # 고품질 리뷰만 남김
        if len(r['review']) < 30:
            filtered_cnt += 1
            # print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # 최대 길이가 50까지 되도로 자름 
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [58]:
# 평가 테스트
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:39<00:00,  3.99s/it]

Wins: 8, Losses: 2, Ties: 0


### 응답결과 고도화 3 - One-Shot Prompting

In [60]:
reviews_1shot, _ = preprocess_reviews(path='./res/3.kyongpo_the_hongc.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-3.5-turbo-0125')
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:05<00:00,  6.53s/it]

Wins: 9, Losses: 1, Ties: 0


In [61]:
summaries

['강릉 여행길에 들러서 예약을 잘한 호텔입니다. 주인의 품격을 느낄 수 있고 조용하고 편안한 분위기가 좋습니다. 고급스러운 로비와 깔끔한 객실로 만족스러운 1박을 보냈습니다. 침구가 훌륭하여 편안한 휴식을 취할 수 있었고, 다음 여행 시 다시 묵을 예정입니다. 객실이 깨끗하고 아늑하며, 침구류의 냄새가 더 없어서 만족하셨던 것 같습니다. 또한 직원분들이 친절하게 안내해주어 편안한 시간을 보낼 수 있었습니다.',
 '강릉 여행길에 들르게 되어 아주 예약을 잘했습니다. 호텔에서는 주인의 품격이 느껴지며 조용하고 좋은 환경이었습니다. 로비도 고급스럽고 객실도 깔끔하여 만족스러운 1박을 했습니다. 침구가 편안해서 잘 쉴 수 있었고, 다음에 여행을 가도 다시 묵고 싶은 곳이었습니다.전체적으로 친절한 안내와 청결함이 편안한 느낌을 주었고, 부모님도 만족하셨다고 합니다.더불어 직원분들의 친절한 응대와 깨끗하고 아늑한 객실이 마음에 들어서 자주 찾게 되는 곳입니다.식음에 대한 안내가 미흡해 아쉬운 점이 있지만, 시설은 깔끔하고 주변 환경이 좋아 다시 방문하고 싶은 곳이었습니다.',
 '강릉 여행길에 들렀는데 아주 예약을 잘한 것 같아요. 호텔이 주인의 품격을 느낄 수 있어요. 조용하고 좋습니다. 로비도 고급스럽고 객실도 깔끔해서 너무 만족스러운 1박이었어요. 침구가 너무 좋아서 편안히 잘 때도 했구요. 다음에 여행을 가면 여기에 또 묵을 거예요. 친절한 안내와 정리정돈이 잘 되어 있어서 편안하게 잘수 있었어요. 부모님께 예약해드렸는데 너무 만족하셨어요! 깨끗하고 아늑한곳이라 자주 방문한답니다. 객실은 깨끗하고 편안하고 전체적으로 좋았어요. 부모님들이 편안히 지내셨다고 하시네요. 하루를 편안히 보낼 수 있었어요. 조식 샌드위치도 진짜 맛있어요!',
 '강릉 여행길에 들러서 아주 예약을 잘 했었어요. 호텔이 주인의 품격이 느껴지며 조용하고 좋았습니다. 로비도 고급스럽고 객실도 깔끔해서 만족스러운 1박을 했습니다. 침구가 너무 좋아서 편안하게 잘 수 있었고, 친절한 안내와 정리

### 응답결과 고도화 3 - one-Shot Prompting

In [62]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:43<00:00,  4.40s/it]

Wins: 7, Losses: 3, Ties: 0


### prompt_1shot을 피클파일로 저장하기

In [ ]:
# prompt_1shot을 피클파일로 저장하기
file_name = "./res/prompt_1shotv1.pickle"
# 파일로 저장
import pickle
with open(file_name, 'wb') as f:
    pickle.dump(prompt_1shot, f)

In [64]:
print(summaries)

['강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안하게 잠을 잘 수 있었고, 다음에 여행을 가면 다시 묵을 의향이 있습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀습니다. 부모님도 만족하셨고, 깨끗하고 아늑한 숙소에서 편안한 시간을 보냈습니다. 객실은 깨끗하고 편안하며, 직원분들도 친절하셨습니다. 전체적으로 좋은 경험이었고, 다음에도 이용할 의향이 있습니다. 위치가 교동에 있어 오가기 편하며, 직원들도 친절하고 깨끗한 숙소였습니다. 주차타워 이용이 불편한 점이 있지만 전반적으로 깔끔하고 만족스러운 경험이었습니다.BindingUtil요원들도 친절하고 좋았으며, 객실도 깨끗하고 특히 조식을 칭찬하는 리뷰가 많았습니다. 숙소가 따듯하고 깨끗하여 만족스러웠고, 다음에도 이용할 예정이라고 칭찬하는 리뷰가 있었습니다.', '강릉 여행길에 있는 호텔은 주인의 품격이 느껴지며 조용하고 좋은 숙소입니다. 로비와 객실 모두 고급스럽고 깔끔하여 만족스러운 1박을 할 수 있었으며, 다음에도 이용할 의향이 있습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀으며, 부모님도 너무 만족하셨습니다. 전체적으로 깨끗하고 편안한 숙소이며, 직원분들도 친절하여 좋은 경험을 한 것 같습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안하게 잠을 잘 수 있었고, 다음에 여행을 가면 다시 묵을 의향이 있습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀습니다. 부모님도 만족하셨고, 깨끗하고 아늑한 숙소에서 편안한 시간을 보냈습니다. 객실은 깨끗하고 편안하며, 직원분들도 친절하셨습니다. 전체적으로 좋은 경험이었고, 다음에도 이용할 의향이 있습니다. (요